In [1]:
%load_ext autoreload
%autoreload 2

import os
import fitz
from utils import process_documents, TextWord
import pdfplumber

base_dir = os.getcwd()

In [2]:
pdf_path = "data/NAB/Berichte_NAB 10-025_Kurzarbeitsprogram Geothermiebohrung Schlattingen.PDF"

In [3]:
document_text = process_documents(pdf_path)
for page_num, text in document_text.items():
    #print(f"Document: {page_num}:\n {text}")

SyntaxError: incomplete input (1314656641.py, line 3)

In [ ]:
## save to json file
import json
output_path = "data/text_files/text_files.json"

with open(output_path, "w", encoding = "utf-8") as file:
    json.dump(document_text, file)

In [7]:
doc = fitz.open(pdf_path)

page_text= {}
for page_index, page in enumerate(doc):
    page_number = page_index + 1
    words = []
    for x0, y0, x1, y1, word, block_no, line_no, _word_no in page.get_text("words"):
        rect = fitz.Rect(x0, y0, x1, y1) * page.rotation_matrix
        text_word = TextWord(rect= rect, text = word, page = page_number)
        words.append(text_word)
       

In [8]:
with pdfplumber.open(pdf_path) as pdf:
    first_page = pdf.pages[0]     
    print(first_page.chars[2])   
print(pdf.metadata["Title"])

{'matrix': (1.0, 0.0, 0.0, 1.0, 248.8011, 646.3766), 'fontname': 'ZNCZOH+NagraDinBold', 'adv': 24.080000000000002, 'upright': True, 'x0': 248.8011, 'y0': 637.5186000000001, 'x1': 272.8811, 'y1': 680.5186000000001, 'width': 24.080000000000013, 'height': 43.0, 'size': 43.0, 'mcid': None, 'tag': 'OC', 'object_type': 'char', 'page_number': 1, 'ncs': 'DeviceCMYK', 'text': 'b', 'stroking_color': None, 'stroking_pattern': None, 'non_stroking_color': (0.0, 0.0, 0.0, 1.0), 'non_stroking_pattern': None, 'top': 161.37139999999988, 'bottom': 204.37139999999988, 'doctop': 161.37139999999988}
Berichtsvorlage
